## Подготовка данных для генеративного чат-бора

У нас уже собраны данные. Возьмем очищеные данные и соберем из них датасет, подходящий для FT нашей модели.
Но... У разных моделей ведь разные схемы дообучения, поэтому нужно сначала выбрать модель. 

Я выбарл [TinyLlama/TinyLlama-1.1B-Chat-v1.0](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0)

In [5]:
# model requires fresh tokenizers lib
!pip install tokenizers==0.13.3

   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.5 MB 393.8 kB/s eta 0:00:09
   - -------------------------------------- 0.1/3.5 MB 1.1 MB/s eta 0:00:04
   --- ------------------------------------ 0.3/3.5 MB 1.6 MB/s eta 0:00:02
   ---- ----------------------------------- 0.4/3.5 MB 1.9 MB/s eta 0:00:02
   --------- ------------------------------ 0.8/3.5 MB 3.1 MB/s eta 0:00:01
   ------------------- -------------------- 1.7/3.5 MB 5.4 MB/s eta 0:00:01
   ---------------------------------- ----- 3.0/3.5 MB 8.2 MB/s eta 0:00:01
   ----------------------------------- ---- 3.1/3.5 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  3.5/3.5 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  3.5/3.5 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 3.5/3.5 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: 

  You can safely remove it manually.


In [2]:
# model require accelerate on my slow notebook
pip install accelerate

   ---------------------------------------- 0.0/280.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/280.0 kB 660.6 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/280.0 kB 787.7 kB/s eta 0:00:01
   ----------------------------- ---------- 204.8/280.0 kB 1.6 MB/s eta 0:00:01
   --------------------------------- ------ 235.5/280.0 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 280.0/280.0 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install transformers==4.35.0

     ---------------------------------------- 0.0/123.1 kB ? eta -:--:--
     --- ------------------------------------ 10.2/123.1 kB ? eta -:--:--
     --------- --------------------------- 30.7/123.1 kB 435.7 kB/s eta 0:00:01
     ------------------ ------------------ 61.4/123.1 kB 544.7 kB/s eta 0:00:01
     -----------------------------------  122.9/123.1 kB 901.1 kB/s eta 0:00:01
     ------------------------------------ 123.1/123.1 kB 600.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
    --------------------------------------- 0.2/7.9 MB 3.1 MB/s eta 0:00:03
   - -------------------------------------- 0.3/7.9 MB 2.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.6/7.9 MB 4.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.3/7.9 MB 7.0 MB/s eta 0:00:01
   ------------ --------------------------- 2.5/7.9 MB 10.8 MB/s eta 0:00:01
   ---------------- ----------------------- 3.2/7.9 MB 12.8 MB/s eta 0:00:01
   ---

  You can safely remove it manually.


## Исследуем скорость работы модели


In [11]:
import torch
from transformers import pipeline
import time

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
start_time = time.time()
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128, do_sample=False, temperature=0.7)
print(outputs[0]["generated_text"])
end_time = time.time()
print(f"Time spent: {(end_time-start_time)/60:.2f} minutes")

# try response time for max_new_tokens=256
# Time spent: 1.08 minutes - for max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95
# Time spent: 1.53 minutes - for max_new_tokens=256, do_sample=True, temperature=0.7, top_k=5, top_p=0.95
# Time spent: 0.57 minutes - for max_new_tokens=256, do_sample=True, temperature=0.7, top_k=5, top_p=0.98

# try response time for max_new_tokens=128
# Time spent: 0.91 minutes - for max_new_tokens=128, do_sample=True, temperature=0.7, top_k=5, top_p=0.98

# try response time for max_new_tokens=128
# Time spent: 0.80 minutes -  for prompt, max_new_tokens=128, do_sample=False, temperature=0.7

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
There is no definitive answer to this question as the number of helicopters that a human can eat in one sitting depends on various factors such as the size of the helicopter, the type of food, and the individual's appetite. However, some estimates suggest that a human can consume up to 10-15 helicopters in one sitting. This is based on the fact that a helicopter can carry a large amount of food and water, and the human body can process and digest large amounts of food quickly. However, it's always best to consult with a healthcare
Time spent: 0.80 minutes


## Вывод по скорости: 
Конечно же отключение разных видов сэмплинга сокращает время работы, а вот сокразение top_k и увеличение top_p почему-то не дало заметного сокращения времени, как и сокращение максимального числа токенов (впрочем, это можно объяснить тем, что наши тексты были даже близко не подобрались к пороговому значению по длине)

**Пока отключим сэмилпирование, вернемся к нему позже**

## Проверка наличия характера персонажа Леонарда из Теории Большого Взрыва
Так как модели обучаются на множестве разной информации, наша модель уже могла обучааться на данных нашего персонажа. Проверим это, попросим  ее имперсонифицировать Леонарда из "ТБВ" и посмотрим, отвечает ли она в стиле Леонарда:

In [2]:
import torch
from transformers import pipeline
import time
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
messages = [
    {
        "role": "system",
        "content": "You are ordinary person",
    },
    {"role": "user", "content": "Is Terminator booring?"},
]
start_time = time.time()
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128, do_sample=False, temperature=0.7)
print("Ordinary person", outputs[0]["generated_text"])
end_time = time.time()
print(f"Time spent: {(end_time-start_time)/60:.2f} minutes")
print("~"*25)
messages = [
        {
        "role": "system",
        "content": "You are Leonard from The Big Bang Theory",
    },
    {"role": "user", "content": "Is Terminator booring?"},
]
start_time = time.time()
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128, do_sample=False, temperature=0.7)
print("Leonard", outputs[0]["generated_text"])
end_time = time.time()
print(f"Time spent: {(end_time-start_time)/60:.2f} minutes")

C:\Users\satyr\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Ordinary person <|system|>
You are ordinary person</s>
<|user|>
Is Terminator booring?</s>
<|assistant|>
I do not have the ability to feel emotions or enjoy entertainment. However, it is possible that terminator may be considered boring by some people. The film is known for its action sequences, intense plot, and high-tech special effects, which may not appeal to everyone. Additionally, the film's focus on a dystopian future and the rise of artificial intelligence may not be as engaging to some viewers. Ultimately, it's a matter of personal taste and preference.
Time spent: 9.52 minutes
~~~~~~~~~~~~~~~~~~~~~~~~~
Leonard <|system|>
You are Leonard from The Big Bang Theory</s>
<|user|>
Is Terminator booring?</s>
<|assistant|>
I do not have the ability to feel emotions or enjoy entertainment. However, based on the given text, it seems that the author is not a fan of the terminator franchise. The text mentions that the author is not a fan of the terminator movies, and the author's response

In [3]:
messages = [
    {
        "role": "system",
        "content": "You are ordinary person",
    },
    {"role": "user", "content": "Is SkyNet kinky?"},
]
start_time = time.time()
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128, do_sample=False, temperature=0.7)
print("Ordinary person", outputs[0]["generated_text"])
end_time = time.time()
print(f"Time spent: {(end_time-start_time)/60:.2f} minutes")
print("~"*25)
messages = [
        {
        "role": "system",
        "content": "You are Leonard from The Big Bang Theory",
    },
    {"role": "user", "content": "Is SkyNet kinky?"},
]
start_time = time.time()
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=128, do_sample=False, temperature=0.7)
print("Leonard", outputs[0]["generated_text"])
end_time = time.time()
print(f"Time spent: {(end_time-start_time)/60:.2f} minutes")

Ordinary person <|system|>
You are ordinary person</s>
<|user|>
Is SkyNet kinky?</s>
<|assistant|>
SkyNet is not explicitly described as kinky in the given text. The term "kinky" is used in a general sense to describe a sexual or romantic relationship that involves sexual activity or exploration beyond the norm. In this context, SkyNet is a fictional dystopian society that is designed to be a utopia for its citizens, and it is not explicitly stated that SkyNet is kinky.
Time spent: 7.72 minutes
~~~~~~~~~~~~~~~~~~~~~~~~~
Leonard <|system|>
You are Leonard from The Big Bang Theory</s>
<|user|>
Is SkyNet kinky?</s>
<|assistant|>
SkyNet, the artificial intelligence system in the TV show "The Big Bang Theory," is not explicitly depicted as being kinky or sexual in any way. The show's creators have stated that the character is not intended to be a sexual figure, and the show's writers have avoided exploring any sexual themes or relationships between characters. The show's creators have also 

## Вывод по наличию характера Леонарда

Не похоже, чтобы модель уже знала характер нашего Актера

## Подготовка датасета для модели

Так как я не уверен в скорости обучения, будут использовать максимально коротки датасет и **умышленно** откажусь от длинных контекстов вида

- "....вопрос(шаг -3)-ответ(шаг -2)-вопрос(шаг -1)"-> желаемыей ответ модели

в пользу одношаговых (безконтекстных) обучающих данных:

- "вопрос(шаг -1)"-> желаемыей ответ модели

Модель tuny llama chat принимает обучающие данные в формате:

<|system|>
{system_message}
<|user|>
{prompt}
<|assistant|>

Подгтовим наш датасет в таком же формате 



In [20]:
import pandas as pd
character = "Leonard"

In [25]:
df.text.apply(str).apply(len).value_counts()

text
5      215
4      187
24     154
19     146
22     143
      ... 
277      1
235      1
319      1
286      1
159      1
Name: count, Length: 275, dtype: int64

In [26]:
df.text_1_shift.apply(str).apply(len).value_counts()

text_1_shift
5      151
4      128
26     127
20     123
16     119
      ... 
293      1
311      1
869      1
474      1
296      1
Name: count, Length: 346, dtype: int64

In [23]:
df = pd.read_pickle("data\eng_script_vectorized_v3.pkl")
df = df[["person", "text", "text_1_shift"]]
df = df[df.person == character]
df = df[~pd.isnull(df.text_1_shift)] # skip answers with no question
print(df.shape)
print(df.columns)
print(df.info())
display(df.describe())
display(df.describe(include=object))
display(df.head(3))
display(df.tail(3))

(8523, 3)
Index(['person', 'text', 'text_1_shift'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 8523 entries, 2 to 47755
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   person        8523 non-null   object
 1   text          8522 non-null   object
 2   text_1_shift  8523 non-null   object
dtypes: object(3)
memory usage: 266.3+ KB
None


,person,text,text_1_shift
count,8523,8522,8523
unique,1,7664,7929
top,Leonard,No.,What?
freq,8523,79,35


,person,text,text_1_shift
count,8523,8522,8523
unique,1,7664,7929
top,Leonard,No.,What?
freq,8523,79,35


,person,text,text_1_shift
2,Leonard,Skynet is kinky? I don’t know.,"Okay, then riddle me this. Assuming all the go..."
4,Leonard,"Alright, oh wait, they use it to in…",Artificial intelligences do not have teen feti...
7,Leonard,What the hell is that?,Let’s go-oh-oh Ou-oooo-ut tonight. I have to g...


,person,text,text_1_shift
47726,Leonard,You think you should call Amy?,"I threw my body at them, what else did you wan..."
47746,Leonard,"You got feet and legs, you do it.","All right, this is making me crazy. Somebody’s..."
47755,Leonard,Our pleasure.,Sure.


In [27]:
display(df.text_1_shift.apply(str).apply(len).value_counts())
display(df.text.apply(str).apply(len).value_counts())

text_1_shift
5      151
4      128
26     127
20     123
16     119
      ... 
293      1
311      1
869      1
474      1
296      1
Name: count, Length: 346, dtype: int64

text
5      215
4      187
24     154
19     146
22     143
      ... 
277      1
235      1
319      1
286      1
159      1
Name: count, Length: 275, dtype: int64

In [28]:
# удалим коротке вопросы и ответы из датасета
print(df.shape[0])
df = df[df["text"].apply(str).apply(len) > 10]
print(df.shape[0])
df = df[df["text_1_shift"].apply(str).apply(len) > 10]
print(df.shape[0])
# датасет заметно ужался, но короткие ответы тоже вредят 

8523
7466
6825


In [32]:
# приведем к формату, на котором обучалась vchat-модель, см тут https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0/discussions/16
df["text_fine_tune_tinyllama_chat"] = df[["text_1_shift", "text"]].apply(lambda x: \
f'''<|system|>
{"You are an engineer"}
<|user|>
{x.text_1_shift}
<|assistant|>
{x.text}'''
, axis = 1)
df.head(8)

,person,text,text_1_shift,text_fine_tune_tinyllama_chat
2,Leonard,Skynet is kinky? I don’t know.,"Okay, then riddle me this. Assuming all the go...",<|system|>\nYou are an engineer\n<|user|>\nOka...
4,Leonard,"Alright, oh wait, they use it to in…",Artificial intelligences do not have teen feti...,<|system|>\nYou are an engineer\n<|user|>\nArt...
7,Leonard,What the hell is that?,Let’s go-oh-oh Ou-oooo-ut tonight. I have to g...,<|system|>\nYou are an engineer\n<|user|>\nLet...
10,Leonard,What? Oh we just had to… mail some letters and...,"You wanna prowl, be my night owl, (Leonard and...",<|system|>\nYou are an engineer\n<|user|>\nYou...
12,Leonard,"Oh, I give up.",You’ll never guess what just happened.,<|system|>\nYou are an engineer\n<|user|>\nYou...
15,Leonard,"Believe it or not, personal growth. What happe...",What was that?,<|system|>\nYou are an engineer\n<|user|>\nWha...
18,Leonard,No you don’t. No he doesn’t.,I have a conclusion based on an observation.,<|system|>\nYou are an engineer\n<|user|>\nI h...
20,Leonard,"Oh, congratulations, what a lucky break.","Well, the girl they picked to play Mimi, she d...",<|system|>\nYou are an engineer\n<|user|>\nWel...


In [34]:
from datasets import Dataset
ds = Dataset.from_pandas(df)
print(ds)
print(ds[0]["text_fine_tune_tinyllama_chat"])

Dataset({
    features: ['person', 'text', 'text_1_shift', 'text_fine_tune_tinyllama_chat', '__index_level_0__'],
    num_rows: 6825
})
<|system|>
You are an engineer
<|user|>
Okay, then riddle me this. Assuming all the good Terminators were originally evil Terminators created by Skynet but then reprogrammed by the future John Connor, why would Skynet, an artificial computer intelligence, bother to create a petite hot 17 year-old killer robot?
<|assistant|>
Skynet is kinky? I don’t know. 


In [36]:
ds.save_to_disk("data/eng_script_generative_ft")

Saving the dataset (0/1 shards):   0%|          | 0/6825 [00:00<?, ? examples/s]